## Problem Description
Determine the optimal volunteer and donation combination to maximize total probable donations. Every volunteer is assigned between 1-2 donors, except for George who will only be assigned 1 donor due to his seniority. Every donor must be assigned exactly volunteer.

Estimated Donation

|Donor 1       |Donor 2       |Donor 3     |Donor 4       |Donor 5    |Donor 6       |
|--------------|--------------|------------|--------------|------------|-------------|
|$2,000,000.00 |$1,200,000.00 |$800,000.00 |$1,550,000.00 |$3,200,000  |$1,800,000.00 |

Probability of donation table

|	    |Donor 1 |Donor 2 |Donor 3 |Donor 4 |Donor 5 |Donor 6 |
|-------|--------|--------|--------|--------|--------|--------|
|George |0.95    |0.9     |0.9     |0.78    |0.78    |0.81    |
|Sarah  |0.89    |0.93    |0.85    |0.82    |0.62    |0.85    |
|Libby	|0.85    |0.91    |0.83    |0.8     |0.5	 |0.83    |
|Eric	|0.75    |0.87    |0.68    |0.65    |0.45	 |0.65    |



In [1]:
import numpy as np 
import pandas as pd
import pulp

In [44]:
donors = ['Donor 1', 'Donor 2', 'Donor 3', 'Donor 4', 'Donor 5', 'Donor 6']
volunteers = ['George', 'Sarah', 'Libby', 'Eric']

donation_amt = [2000000., 1200000., 800000., 1550000., 3200000., 1800000.]

probabilities = np.array([[0.95,0.9,0.9,0.78,0.78,0.81],
                [0.89,0.93,0.85,0.82,0.62,0.85],
                [0.85,0.91,0.83,0.8,0.5,0.83],
                [0.75,0.87,0.68,0.65,0.45,0.65]])

donation_prob = pd.DataFrame(probabilities, columns=donors, index=volunteers)

n_donors = len(donors)
n_volunteers = len(volunteers)

num_donations = 1
min_volunteers = 1
max_volunteers = 2

In [85]:
#Create the model
model = pulp.LpProblem(name='goodway_donation_planning', sense=pulp.LpMaximize)

p_vars = [str(i)+str(j) for i in range(1, n_volunteers+1) for j in range(1, n_donors+1)]

probability_variables = pulp.LpVariable.matrix('P', p_vars,
                                             lowBound=0, cat=pulp.LpBinary)

probs = np.array(probability_variables).reshape(n_volunteers, n_donors)

#The sum of each donor must equal 1
for index, donor in enumerate(donors):
    model.addConstraint(pulp.LpConstraint(
        e=pulp.lpSum(probs[:,index]),
        sense=pulp.LpConstraintEQ,
        name='donor_min_' + donor,
        rhs=1))

#The sum of each volunteer must be between 1 and 2 except for George
for index, vol in enumerate(volunteers):
    if vol == 'George':   
        model.addConstraint(pulp.LpConstraint(
            e=pulp.lpSum(probs[index,:]),
            sense=pulp.LpConstraintEQ,
            name='volunteer_min_' + vol,
            rhs=1))
    else:
        model.addConstraint(pulp.LpConstraint(
            e=pulp.lpSum(probs[index,:]),
            sense=pulp.LpConstraintGE,
            name='volunteer_min_' + vol,
            rhs=1))
        model.addConstraint(pulp.LpConstraint(
            e=pulp.lpSum(probs[index,:]),
            sense=pulp.LpConstraintLE,
            name='volunteer_max_' + vol,
            rhs=2))

objective = pulp.lpSum([probs[i,j]*probabilities[i,j]*donation_amt[j]for i in range(n_volunteers) for j in range(n_donors)])

model.setObjective(objective)

model.solve()

if model.status == 1:
    print(f'status: {model.status}, {pulp.LpStatus[model.status]}')
    print(f'objective: ${model.objective.value():,.0f}')
    output = []
    for i,var in enumerate(model.variables()):
        output.append(var.value())
    output = np.array(output).reshape(n_volunteers, n_donors)

    print(pd.DataFrame(output, columns=donors, index=volunteers))
else:
    print(f'status: {model.status}, {pulp.LpStatus[model.status]}')

status: 1, Optimal
objective: $8,754,000
        Donor 1  Donor 2  Donor 3  Donor 4  Donor 5  Donor 6
George      0.0      0.0      0.0      0.0      1.0      0.0
Sarah       1.0      0.0      0.0      0.0      0.0      1.0
Libby       0.0      0.0      1.0      1.0      0.0      0.0
Eric        0.0      1.0      0.0      0.0      0.0      0.0


In [86]:
model

goodway_donation_planning:
MAXIMIZE
1900000.0*P_11 + 1080000.0*P_12 + 720000.0*P_13 + 1209000.0*P_14 + 2496000.0*P_15 + 1458000.0*P_16 + 1780000.0*P_21 + 1116000.0*P_22 + 680000.0*P_23 + 1271000.0*P_24 + 1984000.0*P_25 + 1530000.0*P_26 + 1700000.0*P_31 + 1092000.0*P_32 + 664000.0*P_33 + 1240000.0*P_34 + 1600000.0*P_35 + 1494000.0*P_36 + 1500000.0*P_41 + 1044000.0*P_42 + 544000.0*P_43 + 1007500.0*P_44 + 1440000.0*P_45 + 1170000.0*P_46 + 0.0
SUBJECT TO
donor_min_Donor_1: P_11 + P_21 + P_31 + P_41 = 1

donor_min_Donor_2: P_12 + P_22 + P_32 + P_42 = 1

donor_min_Donor_3: P_13 + P_23 + P_33 + P_43 = 1

donor_min_Donor_4: P_14 + P_24 + P_34 + P_44 = 1

donor_min_Donor_5: P_15 + P_25 + P_35 + P_45 = 1

donor_min_Donor_6: P_16 + P_26 + P_36 + P_46 = 1

volunteer_min_George: P_11 + P_12 + P_13 + P_14 + P_15 + P_16 = 1

volunteer_min_Sarah: P_21 + P_22 + P_23 + P_24 + P_25 + P_26 >= 1

volunteer_max_Sarah: P_21 + P_22 + P_23 + P_24 + P_25 + P_26 <= 2

volunteer_min_Libby: P_31 + P_32 + P_33 + P_